## Pooled Fund Gap Detection & Enrichment | CCL Interview Process


#### Goals:

- Transform all data into tables
- Clean Data
- Compare Active Funds and Fee Rules
- Identify funds that appear in Active Funds but are missing in Fee Rules
- Fix missing data with info from Billing Repository
- Create a missing Items File with at least
  - fund_code
  - fund_name
  - series
  - Enriched fields from Billing Repository
  - Any status/notes column indicating enrichment gaps or assumptions.
- Create a summary report:
  - Total number of active fund/series combinations
  - Total number covered by fee rules
  - Total number identified as missing



In [63]:
# Import Necessary Libraries
import pandas as pd

In [64]:
# Config Cell:

files_path = '/content/drive/MyDrive/Project/' # Please change it for your source folder

In [65]:
# Load datasets into Pandas
active_funds = pd.read_csv(files_path + 'active_funds.csv')
fee_rules = pd.read_csv(files_path + 'fee_rules.csv')
billing_repository = pd.read_csv(files_path + 'billing_repository.csv')

In [66]:
# Check tables created
display(active_funds.head())
display(fee_rules.head())
display(billing_repository.head())

,fund_code,fund_name,series,currency,as_of_date
0,PFUS.FWCEMARCB3,US FWC EMARCB3,F,USD,2025-12-31
1,PFUS.FWCEMARCB3,US FWC EMARCB3,A,USD,2025-12-31
2,PFUS.FWCEMARCB3,US FWC EMARCB3,a,CAD,2025-12-31
3,PFUS.FWCEMARCB3,US FWC EMARCB3,USD,CAD,2025-12-31
4,PFUS.FWCEMARCB3,US FWC EMARCB3,Premium,USD,2025-12-31


,fund_code,series,fee_rate_bps,fee_basis,billing_currency,fund_series_merged
0,PFUS.FWCEMARCB3,F-H,70,NAV,USD,NaN
1,PFUS.FWCEMARCB3,ALL,70,NAV,USD,NaN
2,PFUS.FWCEMARCB3,Premium,55,AUM,USD,PFUS.FWCEMARCB3[Series=Premium]
3,PFUS.FWCEMARCB3,F,30,AUM,CAD,SERIES=F|FUND=PFUS.FWCEMARCB3
4,PFUS.FWCEMARCB3,usd,60,NAV,CAD,NaN


,invoice_id,instrument_condition,fee_rate_bps,fee_basis,billing_currency,invoice_contact,cost_center,effective_date
0,INV-1001,(( instrument_view.instrument_id = 'pfus.fwcem...,50,AUM,USD,infra-billing@example.com,RC200,2025-07-28
1,INV-1002,(( instrument_view.instrument_id = 'pfus.fwcem...,60,AUM,CAD,core-billing@example.com,RC200,2025-05-23
2,INV-1003,(( instrument_view.instrument_id = 'pfus.fwcem...,60,NAV,USD,intl-billing@example.com,RC500,2024-01-26
3,INV-1004,(( instrument_view.instrument_id = 'pfus.fwcem...,50,AUM,CAD,rec-billing@example.com,RC110,2025-10-20
4,INV-1005,(( instrument_view.instrument_id = 'pfus.fwcem...,65,NAV,CAD,us-billing@example.com,US310,2024-07-31


In [67]:
# Check attributes of each table
print(active_funds.columns.tolist())
print(fee_rules.columns.tolist())
print(billing_repository.columns.tolist())

['fund_code', 'fund_name', 'series', 'currency', 'as_of_date']
['fund_code', 'series', 'fee_rate_bps', 'fee_basis', 'billing_currency', 'fund_series_merged']
['invoice_id', 'instrument_condition', 'fee_rate_bps', 'fee_basis', 'billing_currency', 'invoice_contact', 'cost_center', 'effective_date']


In [68]:
# Check null values of each one

print("\nACTIVE FUNDS TABLE")
print(active_funds.isnull().sum())
print("FEE RULES TABLE")
print(fee_rules.isnull().sum())
print("\nBILLING REPOSITORY TABLE")
print(billing_repository.isnull().sum())


ACTIVE FUNDS TABLE
fund_code      0
fund_name      0
series        97
currency       0
as_of_date     0
dtype: int64
FEE RULES TABLE
fund_code               0
series                114
fee_rate_bps            0
fee_basis               0
billing_currency        0
fund_series_merged    497
dtype: int64

BILLING REPOSITORY TABLE
invoice_id              0
instrument_condition    0
fee_rate_bps            0
fee_basis               0
billing_currency        0
invoice_contact         0
cost_center             0
effective_date          0
dtype: int64


In [69]:
# Standardize sensitivity case by uppercasing everything

active_funds_clean = active_funds.copy()
for col in ['fund_code', 'fund_name', 'series', 'currency']:
    if col in active_funds_clean.columns:
        active_funds_clean[col] = active_funds_clean[col].str.upper().str.strip()


fee_rules_clean = fee_rules.copy()
for col in ['fund_code', 'series', 'fee_basis', 'billing_currency', 'fund_series_merged']:
    if col in fee_rules_clean.columns:
        fee_rules_clean[col] = fee_rules_clean[col].str.upper().str.strip()


billing_repository_clean = billing_repository.copy()
for col in ['instrument_condition', 'fee_basis', 'billing_currency', 'invoice_contact', 'cost_center']:
    if col in billing_repository_clean.columns:
        billing_repository_clean[col] = billing_repository_clean[col].str.upper().str.strip()


In [70]:
print("Check Active Funds Table")
display(active_funds_clean.head())

print("\nCheck Fee Rules Table")
display(fee_rules_clean.head())

print("\nCheck Billing Repository Table")
display(billing_repository_clean.head())

Check Active Funds Table


,fund_code,fund_name,series,currency,as_of_date
0,PFUS.FWCEMARCB3,US FWC EMARCB3,F,USD,2025-12-31
1,PFUS.FWCEMARCB3,US FWC EMARCB3,A,USD,2025-12-31
2,PFUS.FWCEMARCB3,US FWC EMARCB3,A,CAD,2025-12-31
3,PFUS.FWCEMARCB3,US FWC EMARCB3,USD,CAD,2025-12-31
4,PFUS.FWCEMARCB3,US FWC EMARCB3,PREMIUM,USD,2025-12-31



Check Fee Rules Table


,fund_code,series,fee_rate_bps,fee_basis,billing_currency,fund_series_merged
0,PFUS.FWCEMARCB3,F-H,70,NAV,USD,NaN
1,PFUS.FWCEMARCB3,ALL,70,NAV,USD,NaN
2,PFUS.FWCEMARCB3,PREMIUM,55,AUM,USD,PFUS.FWCEMARCB3[SERIES=PREMIUM]
3,PFUS.FWCEMARCB3,F,30,AUM,CAD,SERIES=F|FUND=PFUS.FWCEMARCB3
4,PFUS.FWCEMARCB3,USD,60,NAV,CAD,NaN



Check Billing Repository Table


,invoice_id,instrument_condition,fee_rate_bps,fee_basis,billing_currency,invoice_contact,cost_center,effective_date
0,INV-1001,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PFUS.FWCEM...,50,AUM,USD,INFRA-BILLING@EXAMPLE.COM,RC200,2025-07-28
1,INV-1002,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PFUS.FWCEM...,60,AUM,CAD,CORE-BILLING@EXAMPLE.COM,RC200,2025-05-23
2,INV-1003,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PFUS.FWCEM...,60,NAV,USD,INTL-BILLING@EXAMPLE.COM,RC500,2024-01-26
3,INV-1004,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PFUS.FWCEM...,50,AUM,CAD,REC-BILLING@EXAMPLE.COM,RC110,2025-10-20
4,INV-1005,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PFUS.FWCEM...,65,NAV,CAD,US-BILLING@EXAMPLE.COM,US310,2024-07-31


In [71]:
# Extract 'fund_code' from 'instrument_condition' attribute
billing_repository_clean['fund_code'] = billing_repository_clean['instrument_condition'].str.extract(r"'([^']+)'")[0]

In [72]:
display(billing_repository_clean.head())

,invoice_id,instrument_condition,fee_rate_bps,fee_basis,billing_currency,invoice_contact,cost_center,effective_date,fund_code
0,INV-1001,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PFUS.FWCEM...,50,AUM,USD,INFRA-BILLING@EXAMPLE.COM,RC200,2025-07-28,PFUS.FWCEMARCB3
1,INV-1002,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PFUS.FWCEM...,60,AUM,CAD,CORE-BILLING@EXAMPLE.COM,RC200,2025-05-23,PFUS.FWCEMARCB3
2,INV-1003,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PFUS.FWCEM...,60,NAV,USD,INTL-BILLING@EXAMPLE.COM,RC500,2024-01-26,PFUS.FWCEMARCB3
3,INV-1004,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PFUS.FWCEM...,50,AUM,CAD,REC-BILLING@EXAMPLE.COM,RC110,2025-10-20,PFUS.FWCEMARCB3
4,INV-1005,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PFUS.FWCEM...,65,NAV,CAD,US-BILLING@EXAMPLE.COM,US310,2024-07-31,PFUS.FWCEMARCB3


In [73]:
# Remove duplicates (when all rows match)
active_funds_distinct = active_funds_clean.drop_duplicates()
fee_rules_distinct = fee_rules_clean.drop_duplicates()

In [74]:
active_funds_distinct.size

3675

In [75]:
# Identifies which fund / series records exist in Source A but are missing in Source B

# Create a set from fee_rules
fee_rules_set = set(
    fee_rules_distinct['fund_code'] + '|' + fee_rules_distinct['series']
)

# Create a set from active_funds
active_funds_distinct['fund_series_key'] = active_funds_distinct['fund_code'] + '|' + active_funds_distinct['series']

# Filter active_funds to keep only those NOT in fee_rules
missing_in_fee_rules = active_funds_distinct[
    ~active_funds_distinct['fund_series_key'].isin(fee_rules_set)
].copy()

/tmp/ipython-input-1184998258.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_funds_distinct['fund_series_key'] = active_funds_distinct['fund_code'] + '|' + active_funds_distinct['series']


In [76]:
display(missing_in_fee_rules.head(20))

,fund_code,fund_name,series,currency,as_of_date,fund_series_key
15,PFUS.FWCEMARCB3,US FWC EMARCB3,I2,CAD,2025-12-31,PFUS.FWCEMARCB3|I2
34,PFUS.GLEESG,US GLEESG,F,CAD,2025-12-31,PFUS.GLEESG|F
35,PFUS.GLEESG,US GLEESG,PREMIUM,CAD,2025-12-31,PFUS.GLEESG|PREMIUM
67,PFUS.USDSHORTDUR,US USD SHORTDUR,PREMIUM,USD,2025-12-31,PFUS.USDSHORTDUR|PREMIUM
84,PFUS.BALGROWTH,US BALGROWTH OWTH,F-H,USD,2025-12-31,PFUS.BALGROWTH|F-H
85,PFUS.BALGROWTH,US BALGROWTH OWTH,F-H,CAD,2025-12-31,PFUS.BALGROWTH|F-H
91,PFUS.INTLDIV,US INTLDIVIDEND,F,CAD,2025-12-31,PFUS.INTLDIV|F
94,PFUS.INTLDIV,US INTLDIVIDEND,F,USD,2025-12-31,PFUS.INTLDIV|F
103,PFUS.COREBOND,US COREBOND,F,CAD,2025-12-31,PFUS.COREBOND|F
109,PFUS.COREBOND,US COREBOND,F,USD,2025-12-31,PFUS.COREBOND|F


In [77]:
# Sort billing_repository by 'fund_code', 'effective_date', and 'invoice_id'
billing_repository_sorted = billing_repository_clean.sort_values(
    ['fund_code', 'effective_date', 'invoice_id'],
    ascending=[True, False, False]
)


In [78]:
display(billing_repository_sorted.head())

,invoice_id,instrument_condition,fee_rate_bps,fee_basis,billing_currency,invoice_contact,cost_center,effective_date,fund_code
1602,INV-2603,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PCFCA.SILV...,50,NAV,USD,CORE-BILLING@EXAMPLE.COM,RC120,2024-10-23,PCFCA.SILVER
1626,INV-2627,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PEFUS.LARG...,60,NAV,CAD,CORE-BILLING@EXAMPLE.COM,RC200,2024-12-15,PEFUS.LARGECAP
286,INV-1287,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PEFUS.LARG...,50,NAV,USD,CORE-BILLING@EXAMPLE.COM,RC200,2024-12-15,PEFUS.LARGECAP
1709,INV-2710,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PFBCA.MOME...,60,AUM,CAD,REC-BILLING@EXAMPLE.COM,RC220,2024-10-18,PFBCA.MOMENTUM
1194,INV-2195,(( INSTRUMENT_VIEW.INSTRUMENT_ID = 'PFBCA.MOME...,35,AUM,CAD,REC-BILLING@EXAMPLE.COM,RC220,2024-10-18,PFBCA.MOMENTUM


In [79]:
# Keep only most updated data for each 'fund_code'
billing_most_recent = billing_repository_sorted.drop_duplicates(
    subset=['fund_code'],
    keep='first'
)


In [80]:
# Rename columns to keep track of origin table
billing_for_merge = billing_most_recent[['fund_code', 'fee_rate_bps', 'fee_basis', 'billing_currency', 'invoice_contact', 'cost_center', 'effective_date']].copy()
billing_for_merge.columns = ['fund_code', 'bil_fee_rate_bps', 'bil_fee_basis', 'bil_billing_currency', 'bil_invoice_contact', 'bil_cost_center', 'bil_effective_date']

# Merge on fund_code
missing_in_fee_rules_enriched = missing_in_fee_rules.merge(
    billing_for_merge,
    on='fund_code',
    how='left'
)

In [81]:
display(missing_in_fee_rules_enriched.head())

,fund_code,fund_name,series,currency,as_of_date,fund_series_key,bil_fee_rate_bps,bil_fee_basis,bil_billing_currency,bil_invoice_contact,bil_cost_center,bil_effective_date
0,PFUS.FWCEMARCB3,US FWC EMARCB3,I2,CAD,2025-12-31,PFUS.FWCEMARCB3|I2,65,NAV,CAD,CORE-BILLING@EXAMPLE.COM,RC110,2025-12-26
1,PFUS.GLEESG,US GLEESG,F,CAD,2025-12-31,PFUS.GLEESG|F,60,NAV,CAD,REC-BILLING@EXAMPLE.COM,US310,2025-12-31
2,PFUS.GLEESG,US GLEESG,PREMIUM,CAD,2025-12-31,PFUS.GLEESG|PREMIUM,60,NAV,CAD,REC-BILLING@EXAMPLE.COM,US310,2025-12-31
3,PFUS.USDSHORTDUR,US USD SHORTDUR,PREMIUM,USD,2025-12-31,PFUS.USDSHORTDUR|PREMIUM,45,NAV,USD,US-BILLING@EXAMPLE.COM,US300,2025-12-24
4,PFUS.BALGROWTH,US BALGROWTH OWTH,F-H,USD,2025-12-31,PFUS.BALGROWTH|F-H,45,AUM,CAD,REC-BILLING@EXAMPLE.COM,US300,2025-12-06


In [82]:
# Add Note with decisions made
missing_in_fee_rules_enriched['notes'] = 'Billing data from most recent effective_date; highest invoice_id used when multiple records exist on same date; same billing data applied to all series'

In [84]:
# Check if any row was left behind
print("\nMISSING ITEMS TABLE NULL VALUES")
print(missing_in_fee_rules_enriched.isnull().sum())


MISSING ITEMS TABLE
fund_code               0
fund_name               0
series                  0
currency                0
as_of_date              0
fund_series_key         0
bil_fee_rate_bps        0
bil_fee_basis           0
bil_billing_currency    0
bil_invoice_contact     0
bil_cost_center         0
bil_effective_date      0
notes                   0
dtype: int64


In [83]:
# Save missing items file
output_path = files_path + 'missing_items.csv'
missing_in_fee_rules_enriched.to_csv(output_path, index=False)

print("✅ File saved successfully!")
print(f"\nLocation: {output_path}")

# Summary Report

print("=" * 60)
print("SUMMARY REPORT - ACTIVE FUNDS vs FEE RULES")
print("=" * 60)

total_active = len(active_funds_distinct)
total_in_fee_rules = total_active - len(missing_in_fee_rules)
total_missing = len(missing_in_fee_rules)

print(f"\nTotal active fund/series combinations:        {total_active:>6}")
print(f"Total covered by fee rules:                   {total_in_fee_rules:>6}")
print(f"Total identified as missing:                  {total_missing:>6}")

print("\n" + "-" * 60)

coverage_pct = (total_in_fee_rules / total_active * 100) if total_active > 0 else 0
missing_pct = (total_missing / total_active * 100) if total_active > 0 else 0

print(f"Coverage rate:                                {coverage_pct:>5.1f}%")
print(f"Missing rate:                                 {missing_pct:>5.1f}%")

print("\n" + "=" * 60)

✅ File saved successfully!

Location: /content/drive/MyDrive/Project/missing_items.csv
SUMMARY REPORT - ACTIVE FUNDS vs FEE RULES

Total active fund/series combinations:           735
Total covered by fee rules:                      578
Total identified as missing:                     157

------------------------------------------------------------
Coverage rate:                                 78.6%
Missing rate:                                  21.4%

